In [5]:
import os
from utils import *
import optuna

In [6]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [4]:
study_name = "Easer"
study = optuna.create_study(
    study_name=study_name,
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/database_url.txt'

In [ ]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

def objective(trial):
    topK = trial.suggest_int('topK', 0, 1000)
    l2_norm = trial.suggest_float('l2_norm', 1, 1e10, log=True)
    
    recommender = EASE_R_Recommender(data_train, verbose=False)
    recommender.fit(
        topK=topK if topK > 0 else None,
        l2_norm=l2_norm,
        normalize_matrix=False,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

In [ ]:
study.best_params

{'topK': 72, 'l2_norm': 36.47305040353163}

In [7]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender = EASE_R_Recommender(data_train + data_val, verbose=True)
recommender.fit(topK=72, l2_norm=36.47305040353163)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 18.33 min


In [8]:
submission2(recommender, users, usermap, itemmap, data_train)

TopPopRecommender: URM Detected 224 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 113 ( 0.5%) items with no interactions.
